In [ ]:
from cellshape_voxel.autoencoder import AutoEncoder
from cellshape_voxel.datasets import VoxelDataset
from cellshape_cluster.deep_embedded_clustering import DeepEmbeddedClustering
from cellshape_voxel.autoencoder import AutoEncoder
from cellshape_voxel.datasets import VoxelDataset
import torch
from torch.utils.data import DataLoader
from cellshape_cluster.training_functions import train
from cellshape_voxel.losses import FocalTverskyLoss

In [ ]:
autoencoder  = AutoEncoder(num_layers_encoder=4,
                   num_layers_decoder=4,
                    input_shape=(64, 64, 64, 1),
                   encoder_type='resnet')


model = DeepEmbeddedClustering(autoencoder, 10)

In [ ]:
from torch import nn

PATH_TO_DATASET = '/home/mvries/Documents/Datasets/ModelNet10Voxel/Train'
batch_size = 16
learning_rate = 0.0000001


dataset = VoxelDataset(PATH_TO_DATASET, transform=None, img_size=(64, 64, 64))

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
dataloader_inf = DataLoader(dataset, batch_size=1, shuffle=False)

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate * 16 / batch_size,
    betas=(0.9, 0.999),
    weight_decay=1e-6,
)

recon_crit = FocalTverskyLoss()
clust_crit = nn.KLDivLoss(reduction='sum')

train(model, dataloader, dataloader_inf, 1, optimizer, recon_crit, clust_crit, 1, 0.01, 2, "./")